# Euclidean distance between two images

In [1]:
import ee
ee.Initialize()

In [8]:
from geetools import tools, algorithms

In [9]:
bands = ['B1','B2','B3']

In [10]:
image1 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_232089_20170126').select(bands)

In [11]:
image2 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_232089_20170211').select(bands)

In [12]:
image3 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_232089_20170227').select(bands)

In [13]:
p = ee.Geometry.Point(-72, -42)

In [14]:
data1 = tools.image.getValue(image1, p, 30, side='client')

In [15]:
data1

{'B1': 77, 'B2': 107, 'B3': 290}

In [16]:
data2 = tools.image.getValue(image2, p, 30, side='client')

In [17]:
data2

{'B1': 90, 'B2': 112, 'B3': 272}

In [18]:
data3 = tools.image.getValue(image3, p, 30, side='client')

In [19]:
data3

{'B1': 63, 'B2': 87, 'B3': 193}

In [20]:
distance = algorithms.euclideanDistance(image1, image2)

In [21]:
data_dist = tools.image.getValue(distance, p, 30, side='client')

In [22]:
data_dist

{'distance': 22.759613353482084}

## Compute locally to compare

In [23]:
import math

In [24]:
# Euclidean distance
def distanceF(arr1, arr2):
    zipped = zip(arr1, arr2)
    accum = 0
    for a, b in zipped:
        rest = (a-b)
        calc = rest*rest
        accum += calc
    return math.sqrt(accum)

In [25]:
arr1 = data1.values()
arr1

dict_values([107, 290, 77])

In [26]:
arr2 = data2.values()
arr2

dict_values([112, 272, 90])

In [27]:
local_distance = distanceF(arr1, arr2)

In [28]:
local_distance

22.759613353482084

# Distance sum between one image and a collection of images

In [29]:
col = ee.ImageCollection.fromImages([image2, image3])

In [30]:
summed = algorithms.sumDistance(image1, col)

In [31]:
summed_data = tools.image.getValue(summed, p, 30, side='client')

In [32]:
summed_data

{'sumdist': 122.78461022926308}

## Compute locally to compare results

In [33]:
from copy import copy
# Sum of distances
def sum_distances(values):
    results = {}
    for i, val in enumerate(values):
        val = list(val)
        cop = copy(values)
        cop = [list(a) for a in cop]
        cop.remove(val)
        dist = 0
        for r in cop:
            r = list(r)
            d = distanceF(val, r)
            dist += d
        results[i] = dist

    return results

In [34]:
values = [list(data1.values()), list(data2.values()), list(data3.values())]

In [35]:
values

[[107, 290, 77], [112, 272, 90], [87, 193, 63]]

In [36]:
sum_distances(values)

{0: 122.78461022926308, 1: 109.90891053940321, 2: 187.17429406170214}

## Handling mask values

In [37]:
masked_image = image2.updateMask(image2.gte(0).Not())

In [38]:
data_masked = tools.image.getValue(masked_image, p, 30, side='client')
data_masked

{'B1': None, 'B2': None, 'B3': None}

In [39]:
data1

{'B1': 77, 'B2': 107, 'B3': 290}

In [40]:
distance_masked = algorithms.euclideanDistance(image1, masked_image)

In [41]:
data_dist_masked = tools.image.getValue(distance_masked, p, 30, side='client')
data_dist_masked

{'distance': 318.55611750522075}

In [42]:
distanceF(data1.values(), [0, 0, 0])

318.55611750522075

In [43]:
outside = image1
inside = masked_image

In [44]:
col_masked = ee.ImageCollection.fromImages([inside])

In [45]:
summed_masked = algorithms.sumDistance(outside, col_masked)

In [46]:
summed_data_masked = tools.image.getValue(summed_masked, p, 30, side='client')

In [47]:
summed_data_masked

{'sumdist': 318.55611750522075}